In [74]:
%pip install seaborn
%pip install plotly
%pip install nbformat


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 907.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 6.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import numpy as np
import torch.functional as f
import pandas as pd
odds = np.array([8.0, 4.0, 2.5, 3.0])
probabilities = np.exp(-odds) / np.sum(np.exp(-odds))
probabilities

array([0.00222864, 0.12167987, 0.54533132, 0.33076017])

In [17]:
import itertools as it  

enumerate_combinations = lambda array: list(it.chain(*list((it.combinations(array,n)) for n in range(1,len(array)+1))))

for i in enumerate_combinations(range(len(probabilities))):
    print(i,np.dot(probabilities[list(i)],odds[list(i)]))

(0,) 0.017829155802485914
(1,) 0.486719461734215
(2,) 1.363328307484411
(3,) 0.9922805012921132
(0, 1) 0.5045486175367009
(0, 2) 1.381157463286897
(0, 3) 1.010109657094599
(1, 2) 1.8500477692186261
(1, 3) 1.4789999630263282
(2, 3) 2.355608808776524
(0, 1, 2) 1.867876925021112
(0, 1, 3) 1.496829118828814
(0, 2, 3) 2.37343796457901
(1, 2, 3) 2.8423282705107393
(0, 1, 2, 3) 2.860157426313225


In [26]:
odds = np.array([8.0, 4.0, 2.5, 3.0])
probabilities = np.array([0.002, 0.121, 0.545, 0.330])

kelly_fractions = (odds - 1) * probabilities - (1 - probabilities)
kelly_fractions /= (odds - 1)
# kelly_fractions = np.maximum(kelly_fractions, 0)
kelly_fractions /= np.sum(kelly_fractions)
bet_amounts = 10 * kelly_fractions
bet_amounts

array([ 18.51944793,  22.65997491, -31.83814304,   0.6587202 ])

In [25]:
kelly_fractions

array([-0.14057143, -0.172     ,  0.24166667, -0.005     ])

In [28]:
implied_probabilities = 1 / odds
total_implied_probability = np.sum(implied_probabilities)
normalized_probabilities = implied_probabilities / total_implied_probability
bet_amounts = 10 * normalized_probabilities
# total_implied_probability
bet_amounts

array([1.12781955, 2.2556391 , 3.60902256, 3.0075188 ])

In [29]:
np.sum(bet_amounts)

10.0

In [32]:
bet_amounts += 1.12781955/3
bet_amounts[0] = 0
bet_amounts

array([0.        , 2.63157895, 3.98496241, 3.38345865])

In [33]:
odds * bet_amounts

array([ 0.        , 10.52631579,  9.96240602, 10.15037594])

In [34]:
odds * bet_amounts - bet_amounts

array([0.        , 7.89473684, 5.97744361, 6.76691729])

In [14]:
import seaborn as sns
import numpy as np
import pandas as pd
import itertools as it  

enumerate_combinations = lambda array: list(it.chain(*list((it.combinations(array,n)) for n in range(1,len(array)+1))))

odds = np.array([8.0, 4.0, 2.5, 3.0])
probabilities = np.exp(-odds) / np.sum(np.exp(-odds))
combinations =  enumerate_combinations(range(len(odds)))

budget = 10
bets = np.zeros(shape=(len(combinations),4))
for i,bet in zip(combinations[::-1],bets):
    bet[list(i)] = budget/len(i)
outcomes = bets * odds - budget
cm = sns.light_palette("green", as_cmap=True)

df = pd.concat([pd.DataFrame(bets),
                pd.DataFrame(outcomes)],
                axis=1,keys=['bet_amounts','outcome_profit'])
df[('outcome_profit','best')] = np.max(outcomes,axis=1)
df[('outcome_profit','worst')] = np.min(outcomes,axis=1)
df['expected'] = np.dot(outcomes,probabilities)

def color_negative_positive(col):
    min_val = col.min()
    max_val = col.max()
    
    def compute_color(val, min_val, max_val):
        if pd.isnull(val):
            color = 'white'
        elif val < 0:
            color = f"rgba(255, 125, 125, {min(abs(val) / abs(min_val), 1)})"
        elif val > 0:
            color = f"rgba(125, 255, 125, {min(abs(val) / abs(max_val), 1)})"
        else:
            color = 'white'

        return f'background-color: {color}'
    
    return col.apply(lambda x: compute_color(x, min_val, max_val))

df.style.apply(color_negative_positive)

In [91]:
items = [0, 1, 2, 3, 4]
budget_units = 4
num_options = 4
combinations =  np.array([comb for comb in list(it.product(items, repeat=num_options)) if sum(comb) == budget_units])

budget = 50
odds = np.array([8.0, 4.0, 2.5, 3.0])
probabilities = np.exp(-odds) / np.sum(np.exp(-odds))
bets = combinations * budget/len(odds)
outcomes = bets * odds - budget

df = pd.concat([pd.DataFrame(bets),
                pd.DataFrame(outcomes)],
                axis=1,keys=['bet_amounts','outcome_profit'])
df[('outcome_profit','best')] = np.max(outcomes,axis=1)
df[('outcome_profit','worst')] = np.min(outcomes,axis=1)
df['expected'] = np.dot(outcomes,probabilities)
display(pd.DataFrame(probabilities).T)
df.style.apply(color_negative_positive)



,0,1,2,3
0,0.002229,0.12168,0.545331,0.33076


In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import itertools as it  
pd.options.display.max_rows = None

# odds = np.array([11.0, 6.50, 5.0, 8.0, 6.5, 4.0, 5.5])
# outcome_probabilities = np.exp(-odds) / np.sum(np.exp(-odds))
# budget_units = 5
# num_options = len(odds)
# items = range(budget_units+1)
# combinations =  np.array([comb for comb in list(it.product(items, repeat=num_options)) if sum(comb) == budget_units])

# budget = 50

# bets = combinations * budget/budget_units
# outcomes = bets * odds - budget

# df = pd.concat([pd.DataFrame(bets),
#                 pd.DataFrame(outcomes)],
#                 axis=1,keys=['bet_amounts','outcome_profit'])
# df[('outcome_profit','best')] = np.max(outcomes,axis=1)
# df[('outcome_profit','worst')] = np.min(outcomes,axis=1)
# df['expected'] = np.dot(outcomes,outcome_probabilities)

In [27]:
import seaborn as sns
import numpy as np
import pandas as pd
import itertools as it  
pd.options.display.max_rows = None

def color_negative_positive(col,inverse=False):
    min_val = col.min()
    max_val = col.max()
    
    def compute_color(val, min_val, max_val,inverse=False):
        if pd.isnull(val):
            color = 'white'
        elif val < 0:
            color = f"rgba(255, 125, 125, {min(abs(val) / abs(min_val), 1)})"
            if inverse:
                color = f"rgba(125, 255, 125, {min(abs(val) / abs(max_val), 1)})"
        elif val > 0:
            color = f"rgba(125, 255, 125, {min(abs(val) / abs(max_val), 1)})"
            if inverse:
                color = f"rgba(255, 125, 125, {min(abs(val) / abs(min_val), 1)})"
        else:
            color = 'white'

        return f'background-color: {color}'
    
    return col.apply(lambda x: compute_color(x, min_val, max_val,inverse))


odds = np.array([3.75,2.40,2.50])
# odds = np.array([2.05,1.75])
probabilities = np.exp(-odds) / np.sum(np.exp(-odds))
budget_units = 8
num_options = len(odds)
items = range(budget_units+1)
combinations =  np.array([comb for comb in list(it.product(items, repeat=num_options)) if sum(comb) == budget_units])

budget = 10

bets = combinations * budget/budget_units
outcomes = bets * odds - budget

df = pd.concat([pd.DataFrame(bets),
                pd.DataFrame(outcomes)],
                axis=1,keys=['bet_amounts','outcome_profit'])
df[('outcome_profit','best')] = np.max(outcomes,axis=1)
df[('outcome_profit','worst')] = np.min(outcomes,axis=1)
df['expected'] = np.dot(outcomes,probabilities)

display(pd.DataFrame(odds).T)
display(pd.DataFrame(probabilities).T)

bets_map = bets > 0
num_positions   = pd.Series(bets_map.sum(1),name='num_positions')
prob_winning    = pd.Series(np.dot(bets_map,probabilities),name='prob_winning')
best_winning    = pd.Series([np.ma.masked_array(o,~m,fill_value=np.Inf).max() for o,m in zip(outcomes,bets_map)],name='best_winning')
worst_winning   = pd.Series([np.ma.masked_array(o,~m,fill_value=np.Inf).min() if sum(m) > 1 else pd.NA for o,m in  zip(outcomes,bets_map)],name='worst_winning')
best_winning_cond_prob  = pd.Series([probabilities[np.ma.masked_array(o,~m,fill_value=np.Inf).argmax()]\
                                     /np.ma.masked_array(probabilities,~m,fill_value=np.Inf).sum() for o,m in zip(outcomes,bets_map)]
                                     ,name='best_winning_cond_prob')
worst_winning_cond_prob  = pd.Series([probabilities[np.ma.masked_array(o,~m,fill_value=np.Inf).argmin()]\
                                     /np.ma.masked_array(probabilities,~m,fill_value=np.Inf).sum() if sum(m) > 1 else pd.NA for o,m in zip(outcomes,bets_map)]
                                     ,name='worst_winning_cond_prob')
best_winning_score   = (prob_winning**4*best_winning).rename('best_winning_score')
worst_winning_score   = (prob_winning**4*worst_winning).rename('worst_winning_score')
weighted_winning_score   = (best_winning_score * best_winning_cond_prob + worst_winning_score * worst_winning_cond_prob).rename('weighted_winning_score')

ev_winning      = pd.Series(((bets_map*probabilities)*outcomes/prob_winning.values.reshape(-1,1)).sum(1) / bets_map.sum(1),name='ev_winning')
ev_losing       = pd.Series(-budget,index=ev_winning.index,name='ev_losing')
ev              = pd.Series(ev_winning * prob_winning + ev_losing * (1-prob_winning),name='ev')
df_ev           = pd.concat([df,num_positions,prob_winning,best_winning,worst_winning,best_winning_cond_prob,worst_winning_cond_prob,best_winning_score,worst_winning_score,weighted_winning_score,ev_winning,ev_losing,ev],axis=1)
df_ev.sort_values(['worst_winning','worst_winning_score'],ascending=False).style.apply(color_negative_positive)

,0,1,2
0,3.75,2.4,2.5


,0,1,2
0,0.119792,0.462091,0.418117


,"('bet_amounts', 0)","('bet_amounts', 1)","('bet_amounts', 2)","('outcome_profit', 0)","('outcome_profit', 1)","('outcome_profit', 2)","('outcome_profit', 'best')","('outcome_profit', 'worst')","('expected', '')",num_positions,prob_winning,best_winning,worst_winning,best_winning_cond_prob,worst_winning_cond_prob,best_winning_score,worst_winning_score,weighted_winning_score,ev_winning,ev_losing,ev
29,3.750000,6.250000,0.000000,4.062500,5.000000,-10.000000,5.000000,-10.000000,-1.384059,2,0.581883,5.000000,4.062500,0.794130,0.205870,0.573209,0.465732,0.551083,2.403498,-10,-2.782614
24,3.750000,0.000000,6.250000,4.062500,-10.000000,5.625000,5.625000,-10.000000,-1.782342,2,0.537909,5.625000,4.062500,0.777300,0.222700,0.470933,0.340118,0.441800,2.638516,-10,-3.201624
30,5.000000,0.000000,5.000000,8.750000,-10.000000,2.500000,8.750000,-10.000000,-2.527430,2,0.537909,8.750000,2.500000,0.222700,0.777300,0.732562,0.209303,0.325833,1.945938,-10,-3.574168
4,0.000000,5.000000,5.000000,-10.000000,2.000000,2.500000,2.500000,-10.000000,0.771549,2,0.880208,2.500000,2.000000,0.475021,0.524979,1.500653,1.200522,1.343091,1.118755,-10,-0.213188
34,5.000000,5.000000,0.000000,8.750000,2.000000,-10.000000,8.750000,-10.000000,-2.208803,2,0.581883,8.750000,2.000000,0.205870,0.794130,1.003116,0.229284,0.388593,1.694813,-10,-3.194986
17,2.500000,0.000000,7.500000,-0.625000,-10.000000,8.750000,8.750000,-10.000000,-1.037254,2,0.537909,8.750000,-0.625000,0.777300,0.222700,0.732562,-0.052326,0.557768,3.331093,-10,-2.829080
35,6.250000,0.000000,3.750000,13.437500,-10.000000,-0.625000,13.437500,-10.000000,-3.272518,2,0.537909,13.437500,-0.625000,0.222700,0.777300,1.125006,-0.052326,0.209866,1.253360,-10,-3.946712
23,2.500000,7.500000,0.000000,-0.625000,8.000000,-10.000000,8.000000,-10.000000,-0.559314,2,0.581883,8.000000,-0.625000,0.794130,0.205870,0.917134,-0.071651,0.713573,3.112184,-10,-2.370242
5,0.000000,6.250000,3.750000,-10.000000,5.000000,-0.625000,5.000000,-10.000000,0.851205,2,0.880208,5.000000,-0.625000,0.524979,0.475021,3.001306,-0.375163,1.397413,1.164004,-10,-0.173360
38,6.250000,3.750000,0.000000,13.437500,-1.000000,-10.000000,13.437500,-10.000000,-3.033548,2,0.581883,13.437500,-1.000000,0.205870,0.794130,1.540499,-0.114642,0.226103,0.986127,-10,-3.607358


In [119]:
def color_negative_positive(col,inverse=False):
    min_val = col.min()
    max_val = col.max()
    
    def compute_color(val, min_val, max_val,inverse=False):
        if pd.isnull(val):
            color = 'white'
        elif val < 0:
            color = f"rgba(255, 125, 125, {min(abs(val) / abs(min_val), 1)})"
            if inverse:
                color = f"rgba(125, 255, 125, {min(abs(val) / abs(max_val), 1)})"
        elif val > 0:
            color = f"rgba(125, 255, 125, {min(abs(val) / abs(max_val), 1)})"
            if inverse:
                color = f"rgba(255, 125, 125, {min(abs(val) / abs(min_val), 1)})"
        else:
            color = 'white'

        return f'background-color: {color}'
    
    return col.apply(lambda x: compute_color(x, min_val, max_val,inverse))

def make_odds_table(odds,budget = 10,budget_units = 8):
    budget = 10
    budget_units = 8
    num_options = len(odds)
    items = range(budget_units+1)
    combinations =  np.array([comb for comb in list(it.product(items, repeat=num_options)) if sum(comb) == budget_units])
    bets = combinations * budget/budget_units
    outcomes = bets * odds - budget
    probabilities = np.exp(-odds) / np.sum(np.exp(-odds))
    df = pd.concat([pd.DataFrame(bets,columns=[f'bet_amt_{i}' for i in range(bets.shape[1])]),
                    pd.DataFrame(outcomes,columns=[f'outcome_profit_{i}' for i in range(outcomes.shape[1])])],
                    axis=1)


    bets_map = bets > 0
    profitable_outcomes_map = outcomes >= 0
    profitable_outcomes_masked = np.ma.masked_array(outcomes,~profitable_outcomes_map,fill_value=0)
    profitable_outcomes_probabilities_masked = np.ma.masked_array(np.tile(probabilities,(len(profitable_outcomes_map),1)),~profitable_outcomes_map,fill_value=0)

    loss_outcomes_masked = np.ma.masked_array(outcomes,profitable_outcomes_map,fill_value=0)
    loss_outcomes_probabilities_masked = np.ma.masked_array(np.tile(probabilities,(len(loss_outcomes_masked),1)),profitable_outcomes_map)

    num_positions   = pd.Series(bets_map.sum(1),name='num_positions').fillna(0)
    prob_hit        = pd.Series(np.dot(bets_map,probabilities),name='prob_hit').fillna(0)
    prob_profit     = pd.Series(np.sum(profitable_outcomes_probabilities_masked,axis=1),name='prob_profit').fillna(0)

    weighted_profit  = pd.Series((profitable_outcomes_masked * (profitable_outcomes_probabilities_masked/profitable_outcomes_probabilities_masked.sum(1).reshape(-1,1))).sum(1),name='weighted_profit').fillna(0)
    weighted_loss    = pd.Series((loss_outcomes_masked * (loss_outcomes_probabilities_masked/loss_outcomes_probabilities_masked.sum(1).reshape(-1,1))).sum(1),name='weighted_loss').fillna(0)
    exp_weighted_profit  = (weighted_profit * prob_profit).rename('exp_weighted_profit').fillna(0)
    exp_weighted_loss  = (weighted_loss * (1-prob_profit)).rename('exp_weighted_loss').fillna(0)
    exp_weighted_margin = (exp_weighted_profit + exp_weighted_loss).rename('exp_weighted_margin').fillna(0)
    df              = pd.concat([df,num_positions,prob_hit,prob_profit,weighted_profit,weighted_loss,exp_weighted_profit,exp_weighted_loss,exp_weighted_margin],axis=1)
    return df


# odds = np.array([3.75,2.40,2.50])
# odds = np.array([4.85,4.0,3.0,13.0,5.5,9.5])
# odds = np.array([2.2,1.6,1.22,3.75,4.75,1.15,1.01,11])
# odds = np.array([8.0,7.0,4.0,6.5,3.5,6.0])
odds = np.array([1.83,3.80,4.20])

odds = np.array([2.5,3.6,2.62])
odds = np.array([3.4,3.6,2.05])


df = make_odds_table(odds)
df.sort_values('exp_weighted_margin',ascending=False).style.apply(color_negative_positive)

,bet_amt_0,bet_amt_1,bet_amt_2,outcome_profit_0,outcome_profit_1,outcome_profit_2,num_positions,prob_hit,prob_profit,weighted_profit,weighted_loss,exp_weighted_profit,exp_weighted_loss,exp_weighted_margin
0,0.000000,0.000000,10.000000,-10.000000,-10.000000,10.500000,1,0.679584,0.679584,10.500000,-10.000000,7.135633,-3.204159,3.931474
9,1.250000,0.000000,8.750000,-5.750000,-10.000000,7.937500,2,0.855760,0.679584,7.937500,-7.663206,5.394199,-2.455413,2.938786
1,0.000000,1.250000,8.750000,-10.000000,-5.500000,7.937500,2,0.823824,0.679584,7.937500,-7.974253,5.394199,-2.555077,2.839121
17,2.500000,0.000000,7.500000,-1.500000,-10.000000,5.375000,2,0.855760,0.679584,5.375000,-5.326411,3.652765,-1.706667,1.946098
10,1.250000,1.250000,7.500000,-5.750000,-5.500000,5.375000,3,1.000000,0.679584,5.375000,-5.637458,3.652765,-1.806331,1.846433
2,0.000000,2.500000,7.500000,-10.000000,-1.000000,5.375000,2,0.823824,0.679584,5.375000,-5.948506,3.652765,-1.905996,1.746769
24,3.750000,0.000000,6.250000,2.750000,-10.000000,2.812500,2,0.855760,0.855760,2.799633,-10.000000,2.395813,-1.442403,0.953410
18,2.500000,1.250000,6.250000,-1.500000,-5.500000,2.812500,3,1.000000,0.679584,2.812500,-3.300664,1.911330,-1.057585,0.853745
11,1.250000,2.500000,6.250000,-5.750000,-1.000000,2.812500,3,1.000000,0.679584,2.812500,-3.611711,1.911330,-1.157250,0.754080
3,0.000000,3.750000,6.250000,-10.000000,3.500000,2.812500,2,0.823824,0.823824,2.932872,-10.000000,2.416171,-1.761756,0.654416


In [120]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

fig = px.scatter(
    df,
    x = df['exp_weighted_loss'].abs(),
    y = df['exp_weighted_profit'].abs(),
    color = 'exp_weighted_margin',
    size ='prob_profit',
    hover_data= df.columns,
    labels={ 'exp_weighted_margin': 'Absolute Margin','x': 'Absolute Weighted Risk', 'y': 'Absolute Weighted Reward'},
    title='Risk-Reward Scatterplot',
    width=800,
    height=800,
    size_max=6
)

budget = 10
scale = df[['exp_weighted_loss','exp_weighted_profit']].abs().max(None)
margin = 1
fig.update_xaxes(range=[0-margin,1*(scale + margin)])
fig.update_yaxes(range=[0-margin,scale + margin])
fig.add_shape(type='line', x0=0.5*budget,x1=0.5*budget, y0=0-margin, y1=1*(scale + margin), yref='y', xref='x', line=dict(color='gray', width=1))
fig.add_shape(type='line', x0=0-margin,x1=1*(scale + margin), y0=0.5*budget, y1=0.5*budget, yref='y', xref='x', line=dict(color='gray', width=1))
fig.add_shape(type='line', x0=0-margin, x1=1*(scale + margin), y0=0-margin,y1=1*(scale + margin), yref='y', xref='x', line=dict(color='gray', width=1,dash='dash'))
fig.update_layout(scene=dict(aspectmode='manual'))
fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=1)),dragmode=False,xaxis=dict(fixedrange = True),yaxis=dict(fixedrange = True))

fig.update_traces(marker=dict(sizemode='diameter', line=dict(width=1, color='black')))
fig.show()



In [24]:
df['exp_weighted_loss'].abs().max() 

8.802075067823832